In [1]:
from datasets import load_dataset

/home/joregan/miniconda3/envs/hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pathlib import Path

In [3]:
BASE=Path("/home/chreri/overflow/OverFlow-sardrag/output/nordanvinden")

In [4]:
filenames = [str(x) for x in BASE.glob("**/*.wav")]

In [5]:
dataset = load_dataset("audiofolder", data_files=filenames)

Resolving data files: 100%|██████████| 550/550 [00:00<00:00, 541137.04it/s]


In [6]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

: 

In [ ]:
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="swedish", task="transcribe")


In [ ]:
import torch

In [1]:
SENTENCES = {
    "sent-1": "Nordanvinden och solen tvistade en gång om vem av dem som var starkast.",
    "sent-2": "Just då kom en vandrare vägen fram, insvept i en varm kappa.",
    "sent-3": "De kom då överens om att den som först kunde få vandraren att ta av sig kappan, han skulle anses vara starkare än den andra.",
    "sent-4": "Då blåste nordanvinden så hårt han någonsin kunde, men ju hårdare han blåste, desto tätare svepte vandraren kappan om sig, och till slut gav nordanvinden upp försöket.",
    "sent-5": "Då lät solen sina strålar skina helt varmt och genast tog vandraren av sig kappan, och så var nordanvinden tvungen att erkänna att solen var den starkaste av de två."
}

: 

In [ ]:
from datasets import Audio
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
def map_to_pred(batch):
    audio = batch["audio"]
    path = audio["path"]
    start = path.find("sent-")
    sentence_id = path[start:start+6]
    batch["reference"] = SENTENCES[sentence_id]
    input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
    with torch.no_grad():
        predicted_ids = model.generate(input_features)[0]
    transcription = processor.decode(predicted_ids)
    batch["prediction"] = processor.tokenizer._normalize(transcription)
    return batch

In [ ]:
res = dataset.map(map_to_pred)

In [ ]:
import os
import whisper
from tqdm import tqdm
from pathlib import Path
BASE=Path("/home/chreri/overflow/OverFlow-sardrag/output/nordanvinden")
filenames = [str(x) for x in BASE.glob("**/*.wav")]
num_files = len(filenames)
model = whisper.load_model("large-v3")